In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - Mistral Model (Deployment)

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_pytorch_mistral_deployment.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_mistral_deployment.ipynb">
      <img alt="GitHub logo" src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook demonstrates deploying prebuilt Mistral models in Vertex AI.

### Objective

- Deploy prebuilt [Mistral models](https://huggingface.co/mistralai) with [vLLM](https://github.com/vllm-project/vllm) containers
    - [mistralai/Mistral-7B-v0.1](https://huggingface.co/mistralai/Mistral-7B-v0.1): pretrained generative text model with 7 billion parameters
    - [mistralai/Mistral-7B-Instruct-v0.1](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1): Instruction fine-tuned version of the Mistral-7B-v0.1 generative text model
    - [mistralai/Mistral-7B-Instruct-v0.2](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2): Improved instruction fine-tuned version of Mistral-7B-Instruct-v0.1 supporting 32k context length

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage pricing](https://cloud.google.com/storage/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Run the notebook

In [ ]:
# @title Setup Google Cloud project

# @markdown 1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

# @markdown 2. [Optional] [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs. Set the BUCKET_URI for the experiment environment. The specified Cloud Storage bucket (`BUCKET_URI`) should be located in the same region as where the notebook was launched. Note that a multi-region bucket (eg. "us") is not considered a match for a single region covered by the multi-region range (eg. "us-central1"). If not set, a unique GCS bucket will be created instead.

# Import the necessary packages.
import os
from datetime import datetime
from typing import Tuple

from google.cloud import aiplatform

# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Enable the Vertex AI API and Compute Engine API, if not already.
print("Enabling Vertex AI API and Compute Engine API.")
! gcloud services enable aiplatform.googleapis.com compute.googleapis.com

# Cloud Storage bucket for storing the experiment artifacts.
# A unique GCS bucket will be created for the purpose of this notebook. If you
# prefer using your own GCS bucket, change the value yourself below.
now = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_URI = "gs://"  # @param {type: "string"}
assert BUCKET_URI.startswith("gs://"), "BUCKET_URI must start with `gs://`."

# Gets the default BUCKET_URI and SERVICE_ACCOUNT if they were not specified by the user.

SERVICE_ACCOUNT = None
shell_output = ! gcloud projects describe $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
print("Using this default Service Account:", SERVICE_ACCOUNT)

# Create a unique GCS bucket for this notebook, if not specified by the user.
assert BUCKET_URI.startswith("gs://"), "BUCKET_URI must start with `gs://`."
if BUCKET_URI is None or BUCKET_URI.strip() == "" or BUCKET_URI == "gs://":
    # Create a unique GCS bucket for this notebook, if not specified by the user
    BUCKET_URI = f"gs://{PROJECT_ID}-tmp-{now}"
    ! gsutil mb -l {REGION} {BUCKET_URI}
else:
    BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
    shell_output = ! gsutil ls -Lb {BUCKET_NAME} | grep "Location constraint:" | sed "s/Location constraint://"
    bucket_region = shell_output[0].strip().lower()
    if bucket_region != REGION:
        raise ValueError(
            "Bucket region '%s' is different from notebook region '%s'"
            % (bucket_region, REGION)
        )

print(f"Using this GCS Bucket: {BUCKET_URI}")

# Provision permissions to the SERVICE_ACCOUNT with the GCS bucket
BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin $BUCKET_NAME

! gcloud config set project $PROJECT_ID

# Initialize Vertex AI API.
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

# The pre-built serving docker images with vLLM
VLLM_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-vllm-serve:20240313_0916_RC00"


def get_job_name_with_datetime(prefix: str) -> str:
    """Gets the job name with date time when triggering training or deployment
    jobs in Vertex AI.
    """
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


def deploy_model_vllm(
    model_name: str,
    model_id: str,
    service_account: str,
    machine_type: str = "g2-standard-8",
    accelerator_type: str = "NVIDIA_L4",
    accelerator_count: int = 1,
    max_model_len: int = 4096,
    gpu_memory_utilization=0.9,
    use_openai_server: bool = False,
    use_chat_completions_if_openai_server: bool = False,
) -> Tuple[aiplatform.Model, aiplatform.Endpoint]:
    """Deploys Mistral models with vLLM on Vertex AI.

    Args:
        model_name: Display name of the model.
        model_id: Model ID or path to model weights.
        service_account: Service account for model uploading and deployment.
        machine_type: Deployment machine type.
        accelerator_type: Deployment accelerator type.
        accelerator_count: Number of accelerators to use.
        max_model_len: Maximum model length.
        gpu_memory_utilization: Fraction of GPU memory to be used for the model
            executor.
        use_openai_server: Whether to use the OpenAI-format vLLM model server.
        use_chat_completions_if_openai_server: If the OpenAI model server is
            used, whether to use the chat completion API as opposed to the text
            completion API. The vLLM text completion API mimics the OpenAI text
            completion API:
            https://platform.openai.com/docs/api-reference/completions/create.
            It has two required parameters: the model ID to direct requests to
            and the prompt. The response includes a "choices" field that
            contains the generated text and a "usage" field that contains token
            counts. The vLLM chat completion API mimics the OpenAI chat
            completion API:
            https://platform.openai.com/docs/api-reference/chat/create. It has
            two required parameters: the model ID to direct requests to and
            "messages" which is a sequence of system/user/assistant/tool
            messages that can represent a multi-turn chat conversation. The
            response includes a "choices" field that contains the generated
            message from a role and a "usage" field that contains token counts.

    Returns:
        Model instance and endpoint instance.
    """
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")

    dtype = "bfloat16"
    if accelerator_type in ["NVIDIA_TESLA_T4", "NVIDIA_TESLA_V100"]:
        dtype = "float16"

    vllm_args = [
        "--host=0.0.0.0",
        "--port=7080",
        f"--model=gs://vertex-model-garden-public-us/{model_id}",
        f"--tensor-parallel-size={accelerator_count}",
        "--swap-space=16",
        f"--dtype={dtype}",
        f"--gpu-memory-utilization={gpu_memory_utilization}",
        f"--max-model-len={max_model_len}",
        "--disable-log-stats",
    ]
    serving_env = {
        "MODEL_ID": model_id,
        "DEPLOY_SOURCE": "notebook",
    }
    if use_openai_server:
        if use_chat_completions_if_openai_server:
            serving_container_predict_route = "/v1/chat/completions"
        else:
            serving_container_predict_route = "/v1/completions"
    else:
        serving_container_predict_route = "/generate"
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=VLLM_DOCKER_URI,
        serving_container_command=[
            "python",
            "-m",
            (
                "vllm.entrypoints.openai.api_server"
                if use_openai_server
                else "vllm.entrypoints.api_server"
            ),
        ],
        serving_container_args=vllm_args,
        serving_container_ports=[7080],
        serving_container_predict_route=serving_container_predict_route,
        serving_container_health_route="/health",
        serving_container_environment_variables=serving_env,
    )

    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=service_account,
    )
    return model, endpoint

In [ ]:
# @title Deploy

# @markdown This section deploys the prebuilt Mistral model with [vLLM](https://github.com/vllm-project/vllm) on a Vertex endpoint. It takes 15 minutes to 1 hour to finish depending on the model and the accelerator.

# @markdown Set the model to deploy.

prebuilt_model_id = "mistralai/Mistral-7B-v0.1"  # @param ["mistralai/Mistral-7B-v0.1", "mistralai/Mistral-7B-Instruct-v0.1", "mistralai/Mistral-7B-Instruct-v0.2"]
# Find Vertex AI prediction supported accelerators and regions in
# https://cloud.google.com/vertex-ai/docs/predictions/configure-compute.

accelerator_type = "NVIDIA_L4"  # @param ["NVIDIA_L4", "NVIDIA_TESLA_V100", "NVIDIA_TESLA_T4", "NVIDIA_TESLA_A100"]

if accelerator_type == "NVIDIA_L4":
    machine_type = "g2-standard-8"
    accelerator_count = 1
elif accelerator_type == "NVIDIA_TESLA_V100":
    machine_type = "n1-standard-16"
    accelerator_count = 2
elif accelerator_type == "NVIDIA_TESLA_T4":
    machine_type = "n1-standard-16"
    accelerator_count = 2
elif accelerator_type == "NVIDIA_TESLA_A100":
    machine_type = "a2-highgpu-1g"
    accelerator_count = 1

# Larger setting of `max-model-len` can lead to higher requirements on
# `gpu-memory-utilization` and GPU configuration. Larger setting of
# `gpu-memory-utilization` increases the risk of running out of GPU memory with
# long prompts.
max_model_len = 4096
gpu_memory_utilization = 0.85

model, endpoint = deploy_model_vllm(
    model_name=get_job_name_with_datetime(prefix="mistral-serve-vllm"),
    model_id=prebuilt_model_id,
    service_account=SERVICE_ACCOUNT,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    max_model_len=max_model_len,
    gpu_memory_utilization=gpu_memory_utilization,
    use_openai_server=False,
    use_chat_completions_if_openai_server=False,
)

In [ ]:
# @title Predict

# @markdown Once deployment succeeds, you can send requests to the endpoint with text prompts. Sampling parameters supported by vLLM can be found [here](https://github.com/vllm-project/vllm/blob/2e8e49fce3775e7704d413b2f02da6d7c99525c9/vllm/sampling_params.py#L23-L64).

# Loads an existing endpoint instance using the endpoint name:
# - Using `endpoint_name = endpoint.name` allows us to get the endpoint name of
#   the endpoint `endpoint` created in the cell above.
# - Alternatively, you can set `endpoint_name = "1234567890123456789"` to load
#   an existing endpoint with the ID 1234567890123456789.
# You may uncomment the code below to load an existing endpoint.

# endpoint_name = endpoint.name
# # endpoint_name = ""  # @param {type:"string"}
# aip_endpoint_name = (
#     f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_name}"
# )
# endpoint = aiplatform.Endpoint(aip_endpoint_name)

prompt = "What is a car?"  # @param {type: "string"}
max_tokens = 50  # @param {type:"integer"}
temperature = 1.0  # @param {type:"number"}
top_p = 1.0  # @param {type:"number"}
top_k = 10  # @param {type:"integer"}

instances = [
    {
        "prompt": prompt,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "top_p": top_p,
        "top_k": top_k,
    },
]
response = endpoint.predict(instances=instances)

for prediction in response.predictions:
    print(prediction)

# Reference the following code for using the OpenAI vLLM server.
# import json
# response = endpoint.raw_predict(
#     body=json.dumps({
#         "model": prebuilt_model_id,
#         "prompt": "My favourite condiment is",
#         "n": 1,
#         "max_tokens": 200,
#         "temperature": 1.0,
#         "top_p": 1.0,
#         "top_k": 10,
#     }),
#     headers={"Content-Type": "application/json"},
# )
# print(response.json())

In [ ]:
# @title Clean up resources
# @markdown  Delete the experiment models and endpoints to recycle the resources
# @markdown  and avoid unnecessary continouous charges that may incur.

endpoint.delete(force=True)
model.delete()

delete_bucket = False  # @param {type:"boolean"}
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI